<img src="images/Picture0.png" width=200x />

# Notebook 03 - Sensitivity Analysis

### Covered in this notebook:

### Prerequisites:

### Credits:
* https://www.or-as.be/blog/tsbd_sp
* https://www.gurobi.com/resources/lp-chapter-7-sensitivity-analysis-of-linear-programming-problems/
* https://support.gurobi.com/hc/en-us/community/posts/360063478872-How-to-generate-sensitivity-analysis-report-in-detail
* https://theory.stanford.edu/~trevisan/cs261/lecture06.pdf

In [2]:
import gurobipy as gp
from gurobipy import GRB

## Sensitivity Analysis

Let's take a moment to discuss an important topic to LP problems.  From a human standpoint, finding the solution to our optimization problem is often only the beginning in terms of practical solutions.  We will still have much to discuss sorting out particulars.  What if our material costs are too high for shareholders?  What if a supplier's availability changes?  What if a material we would like to include was deemed too inefficient for optimality?  We will want to know what happens to our solution if we adjust our inputs, and it is useful not to have to run our model again and again to figure these things out.

<strong>Sensitivity analysis</strong> measures the solution's response to changes in the problem: how "sensitive" our solution is.  Gurobi provides us with quite a bit of additional information about our problem; let's discuss some of the relevant pieces.

### Dual Price

The <strong>dual price</strong> or <strong>shadow price</strong> of a constraint refers to the change in the objective value for each unit change in the constraint's right-hand side.  (If we are optimizing $f()$ subject to $g()>=c$, the shadow price gives the change in $f()$ when $c$ is increased by $1$.)  We retrieve shadow price by querying the `Pi` attribute of our constraint; we can query the individual restraint if we gave it a name or assigned it to a variable, or we can retrieve the shadow prices of all constraints in our model in the order they were added by calling `getAttr('Pi')` on the model itself.

<i>(Note for interested individuals: Gurobi computes and can deliver the [dual](https://theory.stanford.edu/~trevisan/cs261/lecture06.pdf) of your program by writing the model to a DLP or DUA file.  More about reading/writing models in Notebook 05.)</i>

### Reduced cost

Suppose our optimal solution occurs when one or more of our variables takes a zero value.  The <strong>reduced cost</strong> informs us of the amount by which the variable's objective function parameter would need to improve for the variable to take a positive value in the optimal solution.

The reduced cost is equivalent to the shadow price of the variable's nonnegativity constraint; that is, if we are optimizing $f()$ subject to $x>= 0$, then the reduced cost gives the change in $f()$ when the constraint is changed to $x>=1$.  If we require $x$ to be present in our optimal solution, how much does this cost our optimality?  And since our problem is linear, we can also think of this as the amount by which the coefficient of $x$ would need to improve (increase or decrease, depending on our objective sense) for $x$ to appear in our solution, to be "cheaper" than the variables which are present already.  The reduced cost of a variable is given by its `RC` attribute.

### Slack

A <strong>binding</strong> constraint is one which is involved in our optimal solution; changing the right-hand side of such a constraint would change the value of the optimal solution.  Not all constraints are necessarily binding, and nonbinding constraints have some <strong>slack</strong>: defined as the difference between the actual usage or production and the imposed limits on usage or production, slack tells us how much we could increase or decrease the constraint's right-hand side before impacting our optimal solution.  It tells us what is "left over" after applying the constraint.  Naturally, slack is zero for binding constraints and nonzero for nonbinding constraints.  A constraint's slack is given by its `Slack` attribute.

Let's analyze the following problem under a sensitivity analysis.

## Exercise: Power Grid

<img src="images/N03_images/powerGrid.jpeg" width=500/>

* minimize
    * the total cost of shipping/transporting/moving
* subject to
    * the total moved out of a source cannot exceed its supply
    * the total moved into a sink must meet its demand
    * there cannot be negative flow

In [3]:
# write model


In [ ]:
# print information (edit this as you like):
for v in m.getVars():
    print(v.varName, '=', v.x)

print()

for v in m.getVars():
    print(v.varName, 'reduced cost =', v.RC)

print()

for c in m.getConstrs():
    print(c.constrName, 'slack =', c.Slack)

print()

for c in m.getConstrs():
    print(c.constrName, 'shadow price =', c.Pi)

print()

print('total expenses:', m.objVal)

### Questions:

#### Shadow price
1. Which of our hubs is the most expensive to serve?
2. Which energy source(s) should Midwest PG invest in to decrease expenses in the future?
3. Historically, Hub 2 has seen an increase in demand of 400 units during football weekends.  What does this increase in demand cost us?

#### Reduced cost
1. Midwest PG wants to upgrade lines to bring down their carriage costs.  Which lines most need to be improved?
2. What would we need to bring their carriage costs down to in order to make those lines competitive?
3. Are there any lines we need to close outright?

#### Slack
1. Are we free to reduce production at any of our locations?
2. If Midwest PG were considering serving a new hub, could we do so on our existing infrastructure?